In [4]:
!autopep8 --in-place --aggressive --aggressive churn_library.py

In [9]:
!pylint churn_library.py

************* Module churn_library
churn_library.py:40:0: W0102: Dangerous default value [] as argument (dangerous-default-value)
churn_library.py:68:0: C0103: Argument name "df" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:117:0: C0103: Argument name "df" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:146:0: C0103: Argument name "df" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:176:4: C0103: Variable name "y" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:179:4: C0103: Variable name "X" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:206:4: C0103: Variable name "X_train" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:206:13: C0103: Variable name "X_test" doesn't conform to snake_case naming style (invalid-name)
churn_library.py:212:0: R0913: Too many arguments (6/5) (too-many-arguments)
churn_library.py:251:0: C0103: